 El desafío consta de 3 ejercicios independientes que van desde análisis exploratorio, machine learning o el diseño de una solución de data science.

 ¿Qué evaluamos?

 El desafío busca evaluar distintos aspectos como:

- Capacidad analitica y exploración de datos
- Visualización de resultados
- Conocimientos de técnicas de generación de features y modelado
- Análisis de performance
- Buenas prácticas de desarrollo
- Diseño e implementación de Machine learning en producción

 Algunas reglas y recomendaciones:
 1. La mayoría de los ejercicios se piden resolver en Jupyter notebooks y te recomendamos subirlas a un repositorio de GitHub público para compartir los resultados.
 2. No dejes de hacernos preguntas sobre cualquier duda con los enunciados. El desafío se analiza de acuerdo al seniority del postulante y teniendo en cuenta también las necesidades particulares de la posición.

1. Explorar las ofertas relámpago, ¿qué insights puedes generar?

- __Descripción__

 En conjunto con el desafío te compartimos un archivo llamado "ofertas_relampago.csv" el cual posee información de los resultados de ofertas del tipo relampago para un periodo de tiempo y un país determinado.

 Estas ofertas en mercadolibre se pueden ver de la siguiente manera:

 Es decir, son ofertas que tienen una duración definida de algunas horas y un porcentaje de unidades (stock) comprometidas.

 El objetivo de este desafío es hacer un EDA sobre estos datos buscando insights sobre este tipo de ofertas.

 Las columnas del dataset son autoexplicativas pero puedes preguntarnos cualquier duda.

- __Entregable__

 El entregable de este desafío es una Jupyter notebook con el EDA.

2. Similitud entre productos:

- __Descripción__

 Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.

 Para este desafío tenemos un dataset "items_titles.csv" que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil

- __Entregable__

 El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset "item_titles_test.csv" generando como output un listado de la forma donde ordenando por score de similitud podamos encontrar los pares de productos más
 similares en nuestro dataset de test.

3. Previsión de falla

- __Descripción__

 Los galpones de Full de mercado libre cuentan con una flota de dispositivos que transmiten diariamente telemetría agregada en varios atributos.

 Las técnicas de mantenimiento predictivo están diseñadas para ayudar a determinar la condición del equipo de mantenimiento en servicio para predecir cuándo se debe realizar el mantenimiento. Este enfoque promete ahorros de costos sobre el mantenimiento preventivo
 de rutina o basado en el tiempo porque las tareas se realizan solo cuando están justificadas.

- __Entregable__

 Tiene la tarea de generar una Jupyter notebook con un modelo predictivo para predecir la probabilidad de falla del dispositivo con el objetivo de bajar los costos del proceso. Como una referencia, una falla de un dispositivo tiene un costo de 1 mientras el costo de un mantenimiento es 0,5. El archivo "full_devices.csv" tiene los valores diários para los 9 atributos de los dispositivos y la columna que está tratando de predecir se llama 'failure' con el valor binario 0 para no fallar y 1 para fallar

In [1]:
import pandas as pd
import os

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
print(os.getcwd())

/content


In [3]:
directory_path = '/content/drive/My Drive/mercado_livre'
os.chdir(directory_path)

In [4]:
os.listdir(directory_path)

['ofertas_relampago.csv',
 'full_devices.csv',
 'items_titles_test.csv',
 'items_titles.csv',
 'Technical Challenge v2.pdf',
 'Desafio_3.ipynb',
 'similar_products_test.csv',
 'Desafio_2.ipynb',
 'Desafio_1.ipynb']

In [5]:
path_ofertas_relampago = f'{directory_path}/ofertas_relampago.csv'

In [6]:
df_ofertas_relampago_orig = pd.read_csv(path_ofertas_relampago, encoding='utf-8')

In [7]:
df_ofertas_relampago_orig.head()

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS
3,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-23 01:36:12+00:00,lightning_deal,15,13,7.03,2.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES
4,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 15:48:12+00:00,lightning_deal,15,0,39.65,15.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES


Verficação de inconsistência na base (nulos)

In [8]:
print('ofertas_relampago_df', df_ofertas_relampago_orig.isnull().sum().sum())

ofertas_relampago_df 85764


In [9]:
print(df_ofertas_relampago_orig.shape)
df = df_ofertas_relampago_orig.copy()

(48746, 13)


# Cap. 0 - Análises Inicias

In [10]:
df.columns

Index(['OFFER_START_DATE', 'OFFER_START_DTTM', 'OFFER_FINISH_DTTM',
       'OFFER_TYPE', 'INVOLVED_STOCK', 'REMAINING_STOCK_AFTER_END',
       'SOLD_AMOUNT', 'SOLD_QUANTITY', 'ORIGIN', 'SHIPPING_PAYMENT_TYPE',
       'DOM_DOMAIN_AGG1', 'VERTICAL', 'DOMAIN_ID'],
      dtype='object')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48746 entries, 0 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OFFER_START_DATE           48746 non-null  object 
 1   OFFER_START_DTTM           48746 non-null  object 
 2   OFFER_FINISH_DTTM          48746 non-null  object 
 3   OFFER_TYPE                 48746 non-null  object 
 4   INVOLVED_STOCK             48746 non-null  int64  
 5   REMAINING_STOCK_AFTER_END  48746 non-null  int64  
 6   SOLD_AMOUNT                24579 non-null  float64
 7   SOLD_QUANTITY              24579 non-null  float64
 8   ORIGIN                     11316 non-null  object 
 9   SHIPPING_PAYMENT_TYPE      48746 non-null  object 
 10  DOM_DOMAIN_AGG1            48746 non-null  object 
 11  VERTICAL                   48746 non-null  object 
 12  DOMAIN_ID                  48746 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usag

# Variáveis de tempo


In [12]:
df['OFFER_START_DATE'] = pd.to_datetime(df['OFFER_START_DATE'])

df['OFFER_START_DTTM'] = pd.to_datetime(df['OFFER_START_DTTM'])
df['OFFER_START_TIME'] = df['OFFER_START_DTTM'].dt.time
df['OFFER_START_HOUR'] = df['OFFER_START_DTTM'].dt.hour
df['WEEKDAY'] = df['OFFER_START_DTTM'].dt.dayofweek
df['WEEKDAY_NAME'] = df['OFFER_START_DTTM'].dt.day_name()

df['OFFER_FINISH_DTTM'] = pd.to_datetime(df['OFFER_FINISH_DTTM'])
df['OFFER_FINISH_DATE'] = df['OFFER_FINISH_DTTM'].dt.date
df['OFFER_FINISH_TIME'] = df['OFFER_FINISH_DTTM'].dt.time
df['OFFER_FINISH_HOUR'] = df['OFFER_FINISH_DTTM'].dt.hour

In [13]:
df_gb_date_count = df.groupby(['OFFER_START_DATE']).agg({'OFFER_START_DTTM':'count'}).rename(columns={'OFFER_START_DTTM':'count'}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_date_count['OFFER_START_DATE'],
                     y=df_gb_date_count['count']))

fig.update_layout(title_text='Quantidade de Ofertas por Dia',
                  xaxis_title='Offer Start Date',
                  yaxis_title='Count')
fig.show()

In [14]:
df_gb_weekday_count = df.groupby(['WEEKDAY', 'WEEKDAY_NAME']).agg({'OFFER_START_DTTM':'count'}).rename(columns={'OFFER_START_DTTM':'count'}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_weekday_count['WEEKDAY_NAME'],
                     y=df_gb_weekday_count['count']))

fig.update_layout(title_text='Quantidade de Ofertas por Dia da Semana',
                  xaxis_title='Dia da Semana',
                  yaxis_title='Count')
fig.show()

In [15]:
df_gb_time_count = df.groupby(['OFFER_START_TIME']).agg({'OFFER_START_DTTM':'count'}).rename(columns={'OFFER_START_DTTM':'count'}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_time_count['OFFER_START_TIME'],
                     y=df_gb_time_count['count']))

fig.update_layout(title_text='Quantidade de Ofertas pelo Horário',
                  xaxis_title='Offer Start Date',
                  yaxis_title='Count')
fig.show()

Existem 3 picos principais para o lancamento de umma oferta, as 7 da manhã, a 13h e as 19h, possivelmente horários onde o trabalhador possa acessar os canais a procura de algo por esta entrando no trabalho, horário de almoço ou saindo do trabalho.

# Variáveis de Preço/Quantidade/Estoque

In [16]:
df[['INVOLVED_STOCK', 'REMAINING_STOCK_AFTER_END', 'SOLD_AMOUNT', 'SOLD_QUANTITY']].describe()

,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY
count,48746.000000,48746.000000,24579.000000,24579.000000
mean,35.007508,30.565216,51.208898,10.851052
std,206.761058,195.813806,175.254414,45.475305
min,1.000000,-192.000000,0.280000,1.000000
25%,5.000000,4.000000,5.730000,1.000000
50%,10.000000,8.000000,12.420000,3.000000
75%,15.000000,15.000000,30.925000,6.000000
max,9000.000000,8635.000000,4836.570000,1646.000000


É possível pereber como os dados estão extremamente achatados, até o 75% a quantidade vendida (SOLD_QUANTITY), o preço vendido (SOLD_AMOUNT) e as variáveis de estoque (INVOLVED_STOCK e REMAINING_STOCK_AFTER_END)

# Variáveis de Segmento de Produtos

In [17]:
df_gb_vertical_count = df.groupby('VERTICAL').agg({'DOMAIN_ID':'count'}).rename(columns={'DOMAIN_ID':'count'}).sort_values('count', ascending=False).reset_index()
df_gb_vertical_count['total'] = df_gb_vertical_count['count'].sum()
df_gb_vertical_count['%total'] = round((df_gb_vertical_count['count'] / df_gb_vertical_count['total']) * 100, 2)
df_gb_vertical_count['%cumsum'] = df_gb_vertical_count['%total'].cumsum()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_vertical_count['VERTICAL'],
                     y=df_gb_vertical_count['count'],
                     name='Quantidade'))

fig.add_trace(go.Scatter(x=df_gb_vertical_count['VERTICAL'],
                         y=df_gb_vertical_count['%cumsum'],
                         name='Quantidade Acumulada (%)',
                         yaxis='y2'))

fig.update_layout(
    title_text='Vertical com mais ofertas',
    xaxis_title='Vertical',
    yaxis=dict(title='Quantidade'),
    yaxis2=dict(title='Quantidade Acumulada (%)', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    )


fig.update_layout()
fig.show()

In [18]:
df_gb_domain_count = df.groupby('DOM_DOMAIN_AGG1').agg({'DOMAIN_ID':'count'}).rename(columns={'DOMAIN_ID':'count'}).sort_values('count', ascending=False).reset_index()
df_gb_domain_count['total'] = df_gb_domain_count['count'].sum()
df_gb_domain_count['%total'] = round((df_gb_domain_count['count'] / df_gb_domain_count['total']) * 100, 2)
df_gb_domain_count['%cumsum'] = df_gb_domain_count['%total'].cumsum()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_domain_count['DOM_DOMAIN_AGG1'],
                     y=df_gb_domain_count['count'],
                     name='Quantidade'))

fig.add_trace(go.Scatter(x=df_gb_domain_count['DOM_DOMAIN_AGG1'],
                         y=df_gb_domain_count['%cumsum'],
                         name='Quantidade Acumulada (%)',
                         yaxis='y2'))

fig.update_layout(
    title_text='Domínios com mais ofertas',
    xaxis_title='Domínio',
    yaxis=dict(title='Quantidade'),
    yaxis2=dict(title='Quantidade Acumulada (%)', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    )


fig.update_layout()
fig.show()

Os dominios que tiveram a maior quantidade de promoções foram em Casa e Decoração (HOME&DECOR) com 13.83% de todas as promoções, seguido por acessório para vestuário (APPAREL ACCESORIES) com 9% ...

In [19]:
df_gb_domain_count2 = df.groupby(['DOM_DOMAIN_AGG1','DOMAIN_ID']).agg({'DOMAIN_ID':'count'}).rename(columns={'DOMAIN_ID':'count'}).sort_values('count', ascending=False).reset_index()
df_gb_domain_count2['count_sum'] = df_gb_domain_count2.groupby('DOM_DOMAIN_AGG1')['count'].transform('sum')
df_gb_domain_count2['%total'] = round((df_gb_domain_count2['count'] / df_gb_domain_count2['count_sum']) * 100, 2)

def pareto_analysis(df_gb_domain_count2, domain):

    df_gb_domain_count2_tmp = df_gb_domain_count2[df_gb_domain_count2['DOM_DOMAIN_AGG1'] == domain]
    df_gb_domain_count2_tmp['count_sum'] = df_gb_domain_count2_tmp['count'].sum()
    df_gb_domain_count2_tmp['%total'] = round((df_gb_domain_count2_tmp['count'] / df_gb_domain_count2_tmp['count_sum']) * 100, 2)
    df_gb_domain_count2_tmp['%cumsum'] = df_gb_domain_count2_tmp['%total'].cumsum()

    fig = go.Figure()
    fig.add_trace(go.Bar(x=df_gb_domain_count2_tmp['DOMAIN_ID'],
                         y=df_gb_domain_count2_tmp['count'],
                         name='Count'))

    fig.add_trace(go.Scatter(x=df_gb_domain_count2_tmp['DOMAIN_ID'],
                             y=df_gb_domain_count2_tmp['%cumsum'],
                             name='% Cumulative Sum',
                             yaxis='y2'))

    fig.update_layout(
        title=f'Pareto Analysis for Domain: {domain}',
        xaxis_title='Domain ID',
        yaxis=dict(title='Count'),
        yaxis2=dict(title='% Cumulative Sum', overlaying='y', side='right'),
        showlegend=False
    )

    fig.show()

In [20]:
df_gb_domain_count['DOM_DOMAIN_AGG1'].unique()

array(['HOME&DECOR', 'APPAREL ACCESORIES', 'APPAREL', 'COMPUTERS',
       'SPORTS', 'PHARMACEUTICS', 'ELECTRONICS', 'PERSONAL CARE',
       'MOBILE', 'AUTOPARTS', 'FOOTWEAR', 'TOOLS AND CONSTRUCTION',
       'TOYS AND GAMES', 'STATIONARY', 'INDUSTRY', 'BEAUTY EQUIPMENT',
       'FOODS', 'PERSONAL HYGIENE', 'BABY', 'MOTOPARTS',
       'BOOKS, MULTIMEDIA & OTHER E!', 'PETS FOOD', 'PARTY', 'CLEANING',
       'SECURITY', 'PET PRODUCTS', 'SUPLEMENTS', 'BATTERIES',
       'VEHICULAR MULTIMEDIA', 'ACC TOOLS', 'DRINKS', 'WHEELS & TIRES',
       'MUSICAL INSTRUMENTS', 'OTHER', 'ANTIQUES & HOBBIES', 'AGRO'],
      dtype=object)

In [21]:
pareto_analysis(df_gb_domain_count2, 'HOME&DECOR')

In [22]:
pareto_analysis(df_gb_domain_count2, 'APPAREL ACCESORIES')

In [23]:
pareto_analysis(df_gb_domain_count2, 'APPAREL')

In [24]:
pareto_analysis(df_gb_domain_count2, 'COMPUTERS')

In [25]:
pareto_analysis(df_gb_domain_count2, 'SPORTS')

In [26]:
pareto_analysis(df_gb_domain_count2, 'PHARMACEUTICS')

In [27]:
pareto_analysis(df_gb_domain_count2, 'ELECTRONICS')

In [28]:
pareto_analysis(df_gb_domain_count2, 'PERSONAL CARE')

Fazendo a abertura dos subdomínios dentro dos dominios que tiveram mais de 3 mil ofertas nesse período é interessante notar como a quantidade de mascaras cirurgicas teve uma quantidade de vendas bastante expressiva em comparação a outros produtos dentro do seu segmento ou mesmo fora.

# Frete

In [29]:
df_frete = df.groupby('SHIPPING_PAYMENT_TYPE').agg({'SOLD_AMOUNT':'sum'}).sort_values('SOLD_AMOUNT', ascending=False).reset_index()
df_frete

,SHIPPING_PAYMENT_TYPE,SOLD_AMOUNT
0,free_shipping,748536.12
1,none,510127.38


In [30]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_frete['SHIPPING_PAYMENT_TYPE'],
                     y=df_frete['SOLD_AMOUNT']))

fig.update_layout(title_text='Frete',
                  xaxis_title='Shipping Payment Type',
                  yaxis_title='Sold Amount')

fig.show()

Como esperado o frete gratuito tem um grande impacto positivo sobre as vendas

# Origin

In [31]:
df['ORIGIN'].value_counts(dropna=False)

,count
ORIGIN,
NaN,37430
A,11316


A variavel tem muitos valores faltantes que não podem ser presumidos para uma imputação.

# Cap. 1 - Eficiência das Promoções

Vamos analisar agora do ponto de vista de conversão de vendas. Dessa forma, criando 5 bins, representando a porcentagem da quantidade de vendas do estoque disponível:

1.   100 - 80% : Altissima
2.   80 - 50% : Alta
3.   50 - 20% : Média
4.   20 - 0% : Baixa
5.   0% : Baixissima

In [32]:
df['STOCK_DIFFERENCE'] = df['INVOLVED_STOCK'] - df['REMAINING_STOCK_AFTER_END']
df['SOLD_PERCENTAGE'] = round(((df['INVOLVED_STOCK'] - df['REMAINING_STOCK_AFTER_END']) / df['INVOLVED_STOCK']) * 100, 2)
df['SOLD_PERCENTAGE'] = df['SOLD_PERCENTAGE'].clip(0, 100)

df['OFFER_SUCCESS_BINS'] = df['SOLD_PERCENTAGE'].apply(lambda x: '100 - 80' if x >= 80 else '80 - 50' if x >= 50 else '50 - 20' if x >= 20 else '20 - 0' if x > 0 else '0')
dict_labels = {'100 - 80': 'Altissima', '80 - 50': 'Alta', '50 - 20': 'Média', '20 - 0': 'Baixa', '0': 'Baixissima'}
df['OFFER_SUCCESS_LABEL'] = df['OFFER_SUCCESS_BINS'].apply(lambda x: dict_labels[x])

df['SOLD_QUANTITY'] = df['SOLD_QUANTITY'].fillna(df['STOCK_DIFFERENCE'])
df['SOLD_AMOUNT'] = df['SOLD_AMOUNT'].fillna(0)
df['REVENUE'] = df['SOLD_AMOUNT'] * df['SOLD_QUANTITY']

print(df.shape)

(48746, 25)


In [33]:
df_gb_date_quantity = df.groupby(['OFFER_START_DATE']).agg({'SOLD_QUANTITY':'sum', 'REVENUE':'sum'}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_date_quantity['OFFER_START_DATE'],
                     y=df_gb_date_quantity['SOLD_QUANTITY']))

fig.update_layout(title_text='Quantidade de Ofertas por Dia',
                  xaxis_title='Dia',
                  yaxis_title='Quantidade Vendida')

fig.show()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_date_quantity['OFFER_START_DATE'],
                     y=df_gb_date_quantity['REVENUE']))

fig.update_layout(title_text='Receita de Ofertas por Dia',
                  xaxis_title='Dia',
                  yaxis_title='Receita')

fig.show()

É possivel ver no grafico acima que há sazonalidade para a quantidade de vendas de produtos.

In [34]:
df_gb_weekday_quantity = df.groupby(['WEEKDAY', 'WEEKDAY_NAME']).agg({'SOLD_QUANTITY':'sum', 'REVENUE':'sum'}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_weekday_quantity['WEEKDAY_NAME'],
                     y=df_gb_weekday_quantity['SOLD_QUANTITY']))

fig.update_layout(title_text='Quantidade de produtos vendidos por Dia da Semana',
                  xaxis_title='Dia da Semana',
                  yaxis_title='Quantidade Vendida')

fig.show()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_weekday_quantity['WEEKDAY_NAME'],
                     y=df_gb_weekday_quantity['REVENUE']))

fig.update_layout(title_text='Receita de produtos vendidos por Dia da Semana',
                  xaxis_title='Dia da Semana',
                  yaxis_title='Receita')

fig.show()

Em contraste com a quantidade de promoções criadas, descrita no capitulo acima, aqui é possivel ver a quantidade de produtos vendidos e nesse ponto é possivel perceber como a influencia dos dias afetam isso. Durante a semana a quantidade de produtos vendidos e maior do que em comparação nos finais de semana.

In [35]:
pivot_table_weekday = pd.pivot_table(df,
                             values='OFFER_SUCCESS_BINS',
                             index='WEEKDAY_NAME',
                             columns='OFFER_SUCCESS_LABEL',
                             aggfunc='count')

pivot_table_weekday['total'] = pivot_table_weekday.sum(axis=1)

for col in pivot_table_weekday.columns:
    pivot_table_weekday[f'%{col}/total'] = round((pivot_table_weekday[col] / pivot_table_weekday['total']) * 100, 2)

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday', 'Saturday']
pivot_table_weekday = pivot_table_weekday.reindex(weekday_order)

pivot_table_weekday

OFFER_SUCCESS_LABEL,Alta,Altissima,Baixa,Baixissima,Média,total,%Alta/total,%Altissima/total,%Baixa/total,%Baixissima/total,%Média/total,%total/total
WEEKDAY_NAME,,,,,,,,,,,,
Monday,320,716,1061,2936,1274,6307,5.07,11.35,16.82,46.55,20.20,100.0
Tuesday,349,868,1171,3545,1389,7322,4.77,11.85,15.99,48.42,18.97,100.0
Wednesday,347,784,1082,3053,1452,6718,5.17,11.67,16.11,45.45,21.61,100.0
Thursday,349,827,1242,3453,1415,7286,4.79,11.35,17.05,47.39,19.42,100.0
Friday,352,793,1242,3896,1490,7773,4.53,10.20,15.98,50.12,19.17,100.0
Sunday,257,595,852,2945,1185,5834,4.41,10.20,14.60,50.48,20.31,100.0
Saturday,290,703,1149,3964,1400,7506,3.86,9.37,15.31,52.81,18.65,100.0


In [36]:
fig = go.Figure()

for label in ['Altissima', 'Alta', 'Média', 'Baixa', 'Baixissima']:

    fig.add_trace(go.Bar(x=pivot_table_weekday.index,
                         y=pivot_table_weekday[f'%{label}/total'], name=label))

fig.update_layout(title_text='Eficiência de Vendas das Ofertas Relâmpago',
                  xaxis_title='Dia da Semana',
                  yaxis_title='Frequência (%)', barmode='stack')
fig.show()

In [37]:
pivot_table_weekday[['%Altissima/total', '%Alta/total', '%Média/total', '%Baixa/total', '%Baixissima/total']].describe()

OFFER_SUCCESS_LABEL,%Altissima/total,%Alta/total,%Média/total,%Baixa/total,%Baixissima/total
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,10.855714,4.657143,19.761429,15.980000,48.745714
std,0.931627,0.442821,1.019680,0.838729,2.550875
min,9.370000,3.860000,18.650000,14.600000,45.450000
25%,10.200000,4.470000,19.070000,15.645000,46.970000
50%,11.350000,4.770000,19.420000,15.990000,48.420000
75%,11.510000,4.930000,20.255000,16.465000,50.300000
max,11.850000,5.170000,21.610000,17.050000,52.810000


A chance de uma promoção ter aderência baixissima varia em torno de 45% durante os dias comerciais enquanto que durante o final de semana ela sobe em torno de 50%.

In [38]:
df_gb_horario_quantity = df.groupby('OFFER_START_TIME').agg({'SOLD_QUANTITY':'sum', 'REVENUE':'sum'}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_horario_quantity['OFFER_START_TIME'],
                     y=df_gb_horario_quantity['SOLD_QUANTITY']))

fig.update_layout(title_text='Quantidade de produtos vendidos por Horário',
                  xaxis_title='Horário',
                  yaxis_title='Quantidade Vendida')

fig.show()


fig = go.Figure()

fig.add_trace(go.Bar(x=df_gb_horario_quantity['OFFER_START_TIME'],
                     y=df_gb_horario_quantity['REVENUE']))

fig.update_layout(title_text='Receita de produtos vendidos por Horário',
                  xaxis_title='Horário',
                  yaxis_title='Receita')

fig.show()

In [39]:
pivot_table_horario = pd.pivot_table(df,
                             values='OFFER_SUCCESS_BINS',
                             index='OFFER_START_TIME',
                             columns='OFFER_SUCCESS_LABEL',
                             aggfunc='count')

pivot_table_horario['total'] = pivot_table_horario.sum(axis=1)

for col in pivot_table_horario.columns:
    pivot_table_horario[f'%{col}/total'] = round((pivot_table_horario[col] / pivot_table_horario['total']) * 100, 2)

pivot_table_horario

OFFER_SUCCESS_LABEL,Alta,Altissima,Baixa,Baixissima,Média,total,%Alta/total,%Altissima/total,%Baixa/total,%Baixissima/total,%Média/total,%total/total
OFFER_START_TIME,,,,,,,,,,,,
00:00:00,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,100.00,NaN,100.0
01:00:00,NaN,NaN,28.0,24.0,2.0,54.0,NaN,NaN,51.85,44.44,3.70,100.0
02:00:00,NaN,NaN,6.0,3.0,NaN,9.0,NaN,NaN,66.67,33.33,NaN,100.0
03:00:00,NaN,NaN,1.0,6.0,1.0,8.0,NaN,NaN,12.50,75.00,12.50,100.0
04:00:00,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,100.00,NaN,100.0
05:00:00,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,100.00,NaN,100.0
06:00:00,NaN,NaN,1.0,9.0,NaN,10.0,NaN,NaN,10.00,90.00,NaN,100.0
07:00:00,437.0,1043.0,1573.0,7043.0,2478.0,12574.0,3.48,8.29,12.51,56.01,19.71,100.0
08:00:00,NaN,NaN,13.0,40.0,2.0,55.0,NaN,NaN,23.64,72.73,3.64,100.0


In [40]:
fig = go.Figure()

for label in ['Altissima', 'Alta', 'Média', 'Baixa', 'Baixissima']:

    fig.add_trace(go.Bar(x=pivot_table_horario.index,
                         y=pivot_table_horario[f'%{label}/total'], name=label))

fig.update_layout(title_text='Eficiência de Vendas das Ofertas Relâmpago',
                  xaxis_title='Horario',
                  yaxis_title='Frequência (%)', barmode='stack')
fig.show()

In [41]:
pivot_table_horario[['%Altissima/total', '%Alta/total', '%Média/total', '%Baixa/total', '%Baixissima/total']].describe()

OFFER_SUCCESS_LABEL,%Altissima/total,%Alta/total,%Média/total,%Baixa/total,%Baixissima/total
count,15.000000,15.000000,17.000000,21.000000,24.000000
mean,7.222000,4.422000,12.312941,27.145714,60.247500
std,3.355393,1.763508,6.585919,14.433567,20.643577
min,2.250000,2.070000,3.370000,10.000000,33.330000
25%,5.215000,3.410000,7.850000,18.580000,45.925000
50%,6.390000,4.260000,11.750000,25.580000,53.160000
75%,8.555000,5.085000,15.040000,30.130000,73.297500
max,14.920000,8.850000,23.260000,66.670000,100.000000


Percebe-se que existe um aumento de eficiência de conversão a 19h e 13h.

In [42]:
df['OFFER_DURATION'] = df['OFFER_FINISH_DTTM'] - df['OFFER_START_DTTM']
df['OFFER_DURATION_DAYS'] = df['OFFER_DURATION'].dt.days
df['OFFER_DURATION_SECONDS'] = (df['OFFER_DURATION_DAYS'] * 24 * 60 * 60) + df['OFFER_DURATION'].dt.seconds
df['OFFER_DURATION_MINUTES'] = (df['OFFER_DURATION_DAYS'] * 24 * 60) + df['OFFER_DURATION'].dt.seconds / 60
df['OFFER_DURATION_HOURS'] = (df['OFFER_DURATION_DAYS'] * 24) + df['OFFER_DURATION'].dt.seconds / 3600

bins = [i * 0.5 for i in range(int(df['OFFER_DURATION_HOURS'].max() * 2) + 2)]
labels = [f'{bins[i]} - {bins[i+1]}' for i in range(len(bins)-1)]
df['OFFER_DURATION_HOURS_BINS'] = pd.cut(df['OFFER_DURATION_HOURS'], bins=bins, labels=labels, right=False)

In [43]:
df['OFFER_DURATION_DAYS'].value_counts()

,count
OFFER_DURATION_DAYS,
0,48745
4,1


De todas as ofertas relampago, todas elas tiveram menos de 1 dia na base com exceção de 1 oferta que ficou ao ar por 4 dias pelo menos.


Além disso, nota-se que a promoção não concretizou nenhuma venda.

In [44]:
df[df['OFFER_DURATION_DAYS'] == 4]

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,OFFER_START_TIME,OFFER_START_HOUR,WEEKDAY,WEEKDAY_NAME,OFFER_FINISH_DATE,OFFER_FINISH_TIME,OFFER_FINISH_HOUR,STOCK_DIFFERENCE,SOLD_PERCENTAGE,OFFER_SUCCESS_BINS,OFFER_SUCCESS_LABEL,REVENUE,OFFER_DURATION,OFFER_DURATION_DAYS,OFFER_DURATION_SECONDS,OFFER_DURATION_MINUTES,OFFER_DURATION_HOURS,OFFER_DURATION_HOURS_BINS
42018,2021-06-05,2021-06-05 13:00:00+00:00,2021-06-09 21:45:09+00:00,lightning_deal,5,5,0.0,0.0,NaN,none,APPAREL ACCESORIES,APP & SPORTS,MLM-SLEEPING_MASKS,13:00:00,13,5,Saturday,2021-06-09,21:45:09,21,0,0.0,0,Baixissima,0.0,4 days 08:45:09,4,377109,6285.15,104.7525,104.5 - 105.0


Para efeitos de análises futuras vou excluir essa promoção, visto que é um outlier em tempo e não foi bem sucedida em vendas.

In [45]:
df = df[df['OFFER_DURATION_DAYS'] != 4]

In [46]:
df_h_bins = df['OFFER_DURATION_HOURS_BINS'].value_counts().reset_index()
df_h_bins = df_h_bins[df_h_bins['count'] != 0]
df_h_bins['%total'] = (df_h_bins['count'] / df_h_bins['count'].sum()) * 100
df_h_bins

,OFFER_DURATION_HOURS_BINS,count,%total
0,6.0 - 6.5,30019,61.583752
1,8.0 - 8.5,7636,15.665196
2,0.0 - 0.5,3947,8.097241
3,7.0 - 7.5,1981,4.064007
4,5.0 - 5.5,887,1.819674
5,5.5 - 6.0,532,1.091394
6,4.5 - 5.0,486,0.997025
7,4.0 - 4.5,471,0.966253
8,3.5 - 4.0,412,0.845215
9,6.5 - 7.0,400,0.820597


In [47]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_h_bins['OFFER_DURATION_HOURS_BINS'],
                     y=df_h_bins['count'],
                     hovertext=df_h_bins['%total'].round(2).astype(str) + '%'))

fig.update_layout(title_text='Duração das Ofertas Relâmpago (Ordenado por horas)',
                  xaxis_title='Duração (horas)',
                  yaxis_title='Frequência',
                  xaxis={'categoryorder':'array', 'categoryarray': df_h_bins['OFFER_DURATION_HOURS_BINS'].sort_values().tolist()}) # Order x-axis

fig.show()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_h_bins['OFFER_DURATION_HOURS_BINS'],
                     y=df_h_bins['count'],
                     hovertext=df_h_bins['%total'].round(2).astype(str) + '%'))

fig.update_layout(title_text='Duração das Ofertas Relâmpago',
                  xaxis_title='Duração (horas)',
                  yaxis_title='Frequência')
fig.show()

fig.show()

fig = go.Figure()

fig.add_trace(go.Bar(x=df_h_bins['OFFER_DURATION_HOURS_BINS'],
                     y=df_h_bins['%total'],
                     hovertext=df_h_bins['count'].round(2).astype(str)))

fig.update_layout(title_text='Duração das Ofertas Relâmpago',
                  xaxis_title='Duração (horas)',
                  yaxis_title='Frequência (%)')
fig.show()

É possível perceber que a maioria das vezes as promoções duram entre 6h a 6h30, (cerca de 61.6%), seguido por 8h a 8h30 (15.7%) e até 30min (8.09%).

In [48]:
df['OFFER_SUCCESS_BINS'].value_counts()

,count
OFFER_SUCCESS_BINS,
0,23791
50 - 20,9605
20 - 0,7799
100 - 80,5286
80 - 50,2264


In [49]:
pivot_table_duration = pd.pivot_table(df,
                             values='OFFER_SUCCESS_BINS',
                             index='OFFER_DURATION_HOURS_BINS',
                             columns='OFFER_SUCCESS_LABEL',
                             aggfunc='count')

pivot_table_duration = pivot_table_duration.loc[:'12.0 - 12.5']
pivot_table_duration['total'] = pivot_table_duration.sum(axis=1)

for col in pivot_table_duration.columns:
    pivot_table_duration[f'%{col}/total'] = round((pivot_table_duration[col] / pivot_table_duration['total']) * 100, 2)

pivot_table_duration

OFFER_SUCCESS_LABEL,Alta,Altissima,Baixa,Baixissima,Média,total,%Alta/total,%Altissima/total,%Baixa/total,%Baixissima/total,%Média/total,%total/total
OFFER_DURATION_HOURS_BINS,,,,,,,,,,,,
0.0 - 0.5,0,78,0,3869,0,3947,0.00,1.98,0.00,98.02,0.00,100.0
0.5 - 1.0,0,149,1,4,0,154,0.00,96.75,0.65,2.60,0.00,100.0
1.0 - 1.5,0,221,1,6,0,228,0.00,96.93,0.44,2.63,0.00,100.0
1.5 - 2.0,0,244,0,1,1,246,0.00,99.19,0.00,0.41,0.41,100.0
2.0 - 2.5,0,322,12,8,0,342,0.00,94.15,3.51,2.34,0.00,100.0
2.5 - 3.0,0,336,0,1,0,337,0.00,99.70,0.00,0.30,0.00,100.0
3.0 - 3.5,0,371,4,7,5,387,0.00,95.87,1.03,1.81,1.29,100.0
3.5 - 4.0,0,412,0,0,0,412,0.00,100.00,0.00,0.00,0.00,100.0
4.0 - 4.5,6,439,16,5,5,471,1.27,93.21,3.40,1.06,1.06,100.0


In [50]:
fig = go.Figure()

for label in ['Altissima', 'Alta', 'Média', 'Baixa', 'Baixissima']:

    fig.add_trace(go.Bar(x=pivot_table_duration.index,
                         y=pivot_table_duration[f'%{label}/total'], name=label))

fig.update_layout(title_text='Eficiência de Vendas das Ofertas Relâmpago',
                  xaxis_title='Duração (horas)',
                  yaxis_title='Frequência (%)', barmode='stack')
fig.show()

In [51]:
pivot_table_duration[['%Altissima/total', '%Alta/total', '%Média/total', '%Baixa/total', '%Baixissima/total']].describe()

OFFER_SUCCESS_LABEL,%Altissima/total,%Alta/total,%Média/total,%Baixa/total,%Baixissima/total
count,24.000000,24.000000,24.000000,24.000000,24.000000
mean,49.080000,1.774167,6.171667,17.594167,25.378333
std,44.703432,3.091167,11.292215,27.887323,32.103303
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.925000,0.000000,0.000000,0.157500,0.665000
50%,41.665000,0.000000,0.000000,6.445000,3.175000
75%,96.795000,2.685000,8.142500,21.587500,43.827500
max,100.000000,11.250000,40.250000,100.000000,100.000000


O sensação de FOMO (Fear of Missing Out) afeta diretamente as vendas, em promoções que tinham duranção alta a quantidade a eficiencia de vendas é Média (20 - 50%) com tendência a Baixa (0 - 20%) ou Baixissima (0%), enquanto que em oferta com tempo entre 30min a 6h geralmente a eficiência é Altissima (80% a 100% do estoque vendido). A excessão desse comportamento é ofertas de meia hora, possivelmente porque não deu tempo dos clientes visualizarem as ofertas, tendo uma eficiência Baixissima.

In [52]:
df_gb_date_success = df.groupby(['OFFER_START_DATE', 'OFFER_SUCCESS_LABEL']).agg({'OFFER_SUCCESS_BINS':'count'}).reset_index()
df_gb_date_success = df_gb_date_success.rename(columns={'OFFER_SUCCESS_BINS':'count'}).sort_values('OFFER_START_DATE')
df_gb_date_success

pv_date_success = pd.pivot_table(df_gb_date_success,
                             values='count',
                             index='OFFER_START_DATE',
                             columns='OFFER_SUCCESS_LABEL',
                             aggfunc='sum')

pv_date_success['total'] = pv_date_success.sum(axis=1)

for col in pv_date_success.columns:
    pv_date_success[f'%{col}/total'] = round((pv_date_success[col] / pv_date_success['total']) * 100, 2)

pv_date_success

OFFER_SUCCESS_LABEL,Alta,Altissima,Baixa,Baixissima,Média,total,%Alta/total,%Altissima/total,%Baixa/total,%Baixissima/total,%Média/total,%total/total
OFFER_START_DATE,,,,,,,,,,,,
2021-06-01,36,50,125,526,157,894,4.03,5.59,13.98,58.84,17.56,100.0
2021-06-02,28,41,77,389,146,681,4.11,6.02,11.31,57.12,21.44,100.0
2021-06-03,18,65,81,215,131,510,3.53,12.75,15.88,42.16,25.69,100.0
2021-06-04,30,54,86,201,95,466,6.44,11.59,18.45,43.13,20.39,100.0
2021-06-05,20,55,81,189,86,431,4.64,12.76,18.79,43.85,19.95,100.0
2021-06-06,18,29,66,207,92,412,4.37,7.04,16.02,50.24,22.33,100.0
2021-06-07,16,47,90,181,92,426,3.76,11.03,21.13,42.49,21.60,100.0
2021-06-08,14,37,72,127,67,317,4.42,11.67,22.71,40.06,21.14,100.0
2021-06-09,18,48,71,149,80,366,4.92,13.11,19.40,40.71,21.86,100.0


In [53]:
fig = go.Figure()

for label in ['Altissima', 'Alta', 'Média', 'Baixa', 'Baixissima']:

    fig.add_trace(go.Bar(x=pv_date_success.index,
                         y=pv_date_success[f'%{label}/total'], name=label))

fig.update_layout(barmode='stack')

fig.show()

In [54]:
pv_date_success[['%Altissima/total', '%Alta/total', '%Média/total', '%Baixa/total', '%Baixissima/total']].describe()

OFFER_SUCCESS_LABEL,%Altissima/total,%Alta/total,%Média/total,%Baixa/total,%Baixissima/total
count,61.000000,61.000000,61.000000,61.000000,61.000000
mean,11.011475,4.673115,19.812787,16.350328,48.152459
std,2.500730,0.865770,2.240471,2.964198,5.841800
min,5.590000,2.600000,13.890000,11.310000,30.710000
25%,9.160000,4.030000,18.650000,14.650000,43.850000
50%,11.590000,4.550000,19.940000,16.090000,48.430000
75%,12.760000,5.180000,21.170000,17.280000,51.780000
max,16.420000,6.600000,25.690000,30.710000,59.460000


# Conclusão

A análise exploratória dos dados de ofertas relâmpago revelou padrões chave sobre os fatores que influenciam a conversão de vendas. Foi identificado que a duração ideal das ofertas é de 6 horas, período durante o qual a taxa de conversão atinge seu pico. Esse resultado sugere que uma janela de tempo relativamente curta mantém o senso de urgência, sem sobrecarregar os consumidores, que podem se sentir mais inclinados a aproveitar a oferta antes que ela expire. Além disso, ao observar os horários das ofertas, constatamos que os momentos mais eficazes para o lançamento das promoções ocorreram durante o horário de almoço e nas transições de entrada e saída do expediente comercial. Esses períodos coincidem com os momentos em que os consumidores têm maior disponibilidade e tempo para realizar compras, sugerindo que as promoções nesses horários geram mais engajamento e conversão.

A sazonalidade das vendas também foi um fator relevante durante a análise. A conversão foi significativamente mais alta durante os dias úteis, enquanto nos finais de semana houve uma queda substancial, aumentando o número de promoções sem conversão. Esse comportamento pode estar relacionado aos padrões de consumo dos consumidores, que tendem a ser mais focados e produtivos durante a semana, enquanto no fim de semana estão mais dispersos ou menos inclinados a realizar compras rápidas e impulsivas.

Outro ponto crucial para o sucesso das ofertas foi o frete grátis. Ao analisar as ofertas com e sem essa condição, foi claramente visível que o frete grátis teve um impacto direto na conversão de vendas. As ofertas que incluíam frete grátis apresentaram taxas de conversão muito superiores àquelas que não ofereciam essa vantagem. Isso pode ser atribuído ao fato de que os consumidores consideram o custo do frete como um obstáculo adicional, principalmente em compras de valor mais baixo, onde o frete acaba sendo um fator decisivo. O frete grátis, portanto, não apenas elimina esse custo extra para o consumidor, mas também cria uma percepção de valor agregado, tornando a compra mais atraente e acessível. Estratégias que oferecem o frete grátis, seja com um valor mínimo de compra ou em promoções específicas, podem ser mais eficazes para aumentar as vendas e melhorar o desempenho das campanhas.

Além disso, ao analisar as categorias de produtos, verificou-se que Home & Decor, Apparel Accessories, Apparel, Computers, Sports, Pharmaceutics, Electronics e Personal Care foram as mais comuns em termos de volume de ofertas. Dentre elas, a categoria Pharmaceutics se destacou por registrar o maior volume de vendas, o que pode ser atribuído ao contexto da pandemia. Durante esse período, houve um aumento significativo na demanda por produtos farmacêuticos, como medicamentos e itens de saúde, devido à maior preocupação com o bem-estar e à busca por cuidados preventivos e tratamentos. Esse fenômeno reflete como fatores externos, como a pandemia, podem impactar diretamente os padrões de consumo e alterar o desempenho de diferentes categorias de produtos.

Esses insights fornecem informações valiosas para a otimização de futuras campanhas de ofertas relâmpago. Entender quais são os períodos ideais para lançamentos, como a duração e o frete grátis impactam a conversão, e como o contexto externo pode afetar a demanda por categorias específicas de produtos permite um planejamento mais estratégico. Ao aplicar esses aprendizados, é possível maximizar a eficácia das promoções, gerar um aumento nas vendas e proporcionar uma experiência mais satisfatória para os consumidores.